### Из-за небольшого времени на реализацию проекта я бы не успела протестировать и обучить с нуля две различные архитектуры (детекция и многоклассовая классификация). Больший интерес у меня вызвала именно классификация, поэтому я использовала предобученную модель yolon8 для детекции собак на изображениях, чтобы освободить побольше времени на работу с другими архитектурами


In [ ]:
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm

MODEL_PATH = "yolov8n.pt"          # Модель YOLO для детекции
SOURCE_ROOT = "C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/Dog Breeds Image Dataset"        
OUTPUT_ROOT = "C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/Dog Breeds Image Dataset/cropped_dogs"        
CONFIDENCE_THRESH = 0.5            # Порог уверенности (0-1)

model = YOLO(MODEL_PATH)

for breed_dir in tqdm(os.listdir(SOURCE_ROOT)):
    breed_path = os.path.join(SOURCE_ROOT, breed_dir)
    
    # Пропускаем файлы (только папки с породами)
    if not os.path.isdir(breed_path):
        continue
    
    # Создаем папку для обрезанных собак этой породы
    output_breed_dir = os.path.join(OUTPUT_ROOT, breed_dir)
    os.makedirs(output_breed_dir, exist_ok=True)
    
    # Обработка всех фото в папке породы
    for img_name in os.listdir(breed_path):
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        img_path = os.path.join(breed_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Детекция собак (класс 'dog' в COCO = 16)
        results = model.predict(img, classes=[16], conf=CONFIDENCE_THRESH)
        
        # Сохранение каждой найденной собаки
        for i, box in enumerate(results[0].boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            dog_crop = img[y1:y2, x1:x2]
            
            # Сохраняем в папку породы с оригинальным именем + индексом
            output_path = os.path.join(
                output_breed_dir, 
                f"{os.path.splitext(img_name)[0]}_{i}.jpg"
            )
            cv2.imwrite(output_path, dog_crop)